In [1]:
import os, shutil
import random
import tensorflow as tf
from tensorflow.keras import (models, layers, optimizers)

import plotly.express as ex
import plotly.graph_objects as go

2023-01-27 18:52:45.063288: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 18:52:45.171714: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-27 18:52:45.174515: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-27 18:52:45.174535: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
data_path = "../data/processed"

In [3]:
print(len(os.listdir(data_path + "/train/dogs")))
print(len(os.listdir(data_path + "/train/cats")))

1414
1422


In [4]:
IMG_SIZE = 150

# https://www.tensorflow.org/guide/keras/preprocessing_layers
data_augmentation = tf.keras.Sequential([
    layers.Resizing(IMG_SIZE, IMG_SIZE),
    layers.Rescaling(1./255),
    layers.RandomRotation(.4),
    layers.RandomWidth(.2),
    layers.RandomHeight(.2),
    # layers.RandomCrop(.2, .2),
    layers.RandomZoom(.2),
    layers.RandomFlip("horizontal"),
    layers.Resizing(IMG_SIZE, IMG_SIZE),
])

2023-01-27 18:52:46.710019: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-27 18:52:46.710040: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-27 18:52:46.710056: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (hope): /proc/driver/nvidia/version does not exist
2023-01-27 18:52:46.710351: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model = models.Sequential([data_augmentation])
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.build(input_shape=(None, IMG_SIZE, IMG_SIZE, 3))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 150, 150, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)      

In [6]:
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=['acc']
)

In [7]:
batch_size = 16
x_train, x_valid = tf.keras.utils.image_dataset_from_directory(
    (data_path + "/train"),
    batch_size=batch_size,
    image_size=(IMG_SIZE, IMG_SIZE),
    validation_split=.2,
    subset="both",
    seed=42
)

Found 2836 files belonging to 2 classes.
Using 2269 files for training.
Using 567 files for validation.


In [8]:
x_train_count = len(x_train) * batch_size
x_valid_count = len(x_valid) * batch_size
print(x_train_count)
print(x_valid_count)

2272
576


In [9]:
history = model.fit(
    x_train.repeat(),
    steps_per_epoch=int(x_train_count/batch_size),
    epochs=30,
    validation_data=x_valid.repeat(),
    validation_steps=int(x_valid_count/batch_size)
)

Epoch 1/30
142/142 [==============================] - 21s 132ms/step - loss: 0.6952 - acc: 0.5024 - val_loss: 0.6926 - val_acc: 0.4956
Epoch 2/30
142/142 [==============================] - 20s 141ms/step - loss: 0.6926 - acc: 0.5059 - val_loss: 0.6882 - val_acc: 0.5802
Epoch 3/30
142/142 [==============================] - 20s 137ms/step - loss: 0.6891 - acc: 0.5381 - val_loss: 0.6787 - val_acc: 0.5573
Epoch 4/30
142/142 [==============================] - 20s 144ms/step - loss: 0.6804 - acc: 0.5672 - val_loss: 0.6866 - val_acc: 0.5450
Epoch 5/30
142/142 [==============================] - 20s 139ms/step - loss: 0.6660 - acc: 0.5941 - val_loss: 0.7100 - val_acc: 0.5450
Epoch 6/30
142/142 [==============================] - 20s 141ms/step - loss: 0.6539 - acc: 0.6166 - val_loss: 0.6609 - val_acc: 0.6014
Epoch 7/30
142/142 [==============================] - 20s 141ms/step - loss: 0.6478 - acc: 0.6267 - val_loss: 0.6557 - val_acc: 0.6226
Epoch 8/30
142/142 [==============================] - 2

In [10]:
# save model
model.save("./model.h5")

In [11]:
# load model
# model = models.load_model("./model.h5")

In [12]:
acc = history.history['acc']
vall_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = [x for x in range(1, len(acc) + 1)]

fig = go.Figure()
fig.add_trace(go.Scatter(x=epochs, y=acc, name="Training Accuracy"))
fig.add_trace(go.Scatter(x=epochs, y=vall_acc, name="Validation Accuracy"))
fig.update_layout(
    title="Training and Validation Accuracy",
    xaxis_title="Epochs",
    yaxis_title="Accuracy"
)
fig.show()

In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=epochs, y=loss, name="Training Loss"))
fig.add_trace(go.Scatter(x=epochs, y=val_loss, name="Validation Loss"))
fig.update_layout(
    title="Training and Validation Loss",
    xaxis_title="Epochs",
    yaxis_title="Loss"
)
fig.show()